In [2]:
from faker import Faker
import numpy as np
import pandas as pd
import random as rd
from datetime import date
import math

In [19]:
products_labels = ["id", "name", "price", "breeds", "is_for_dog", "is_for_cat"]
locals_vets_labels = ["id", "name", "has_fb", "has_ig", "lat", "lng","is_vet","is_suscribed"]
sales_local_labels = ["id", "id_local", "id_client",
"name_client", "product_id", "product_name", "quantity",
"price", "total_price", "sale_by_publisher", "sale_online","date",'is_suscribed']
# service_sales_local_labels = ["id", "id_local_vet", "id_client",
# "name_client", "id_service", "service_name",
# "price", "sale_by_publisher", "sale_online"]
services_vet_labels = ["id", "id_local_vet",
"name_pet", "breed", "price", "date",'is_online','is_cat','is_dog','service_name','sale_by_publisher',
'name_client','is_suscribed']


In [20]:
Faker.seed(0)
faker= Faker(['en-US'])
rd.seed(0)
sample_num=50

In [21]:
df_dog_breed=pd.read_csv('./dog_breed.csv')
df_cat_breed=pd.read_csv('./cats.csv')

In [22]:
dog_breeds=df_dog_breed.drop_duplicates(subset='breed').sample(sample_num).breed.to_list()
cat_breeds=df_cat_breed.drop_duplicates(subset='breed').sample(sample_num).breed.to_list()

In [23]:
pet_food=pd.read_csv('./openfoodfacts_export (1).csv',sep='\t')
pet_food=pet_food[["code","product_name_en","categories_tags"]]
pet_food.dropna(inplace=True)
pet_food["is_dog_cat"]=pet_food.categories_tags.apply(lambda x: 1 if "dog" in x else (2 if 'cat' in x else rd.randint(1,2)) )

In [24]:
d={
    'service_name':['Peluquería','Cortar uñas','Operación de pata','Baño médico','Desparasitación','Vacunación','servicio general','castración'],
    'prices':[30,20,1000,40,30,50,20,80]
}
df_services=pd.DataFrame(d)

In [25]:
def generate_products(quantity,df_products,dog_breeds,cat_breeds,range_price,products_labels):
    list_products=[]
    for i in range(quantity):
        sample=df_products.sample()
        code=int(sample['code'])
        name=sample['product_name_en'].values[0]
        is_dog_cat=sample['is_dog_cat'].values[0]
        is_for_cat=0
        is_for_dog=0
        breed=''
        if is_dog_cat==1:
            is_for_dog=1
            breed=rd.choice(dog_breeds)
        else:
            is_for_cat=1
            breed=rd.choice(cat_breeds)
        price=rd.random()*rd.randint(*range_price)
        list_products.append((code,name,price,breed,is_for_cat,is_for_dog))
    return pd.DataFrame(list_products,columns=products_labels)
        


In [26]:
def generate_locals(quantity,local_labels):
    list_locals=[]
    for i in range(quantity):
        id_local=i
        name_local=faker.company()
        has_ig,has_fb=faker.random_choices(elements=(1,0),length=2)
        geo_local=faker.local_latlng(country_code="EC")
        lat=geo_local[0]
        lng=geo_local[1]
        is_vet=rd.randint(0,1)
        is_suscribed=rd.randint(0,1)
        list_locals.append((id_local,name_local,has_fb,has_ig,lat,lng,is_vet,is_suscribed))
    return pd.DataFrame(list_locals, columns=local_labels)

In [27]:
def generate_sales_locals(quantity,df_locals,df_products,label_sales,range_date):
    list_sales=[]
    list_names_clients=[faker.name() for i in range (math.ceil(quantity*0.85))]
    for i in range(quantity):
        index_local=rd.randint(0,df_locals.shape[0]-1)
        index_products=rd.randint(0,df_products.shape[0]-1)
        local=df_locals.iloc[index_local]
        product=df_products.iloc[index_products]
        id_local=local.id
        id_client=faker.ssn()
        name_cliente=rd.choice(list_names_clients)
        product_id=product.id
        product_name=product.name
        quantity_value=rd.randint(1,10)
        price=product.price
        total_price=price*quantity_value
        sale_by_publisher=rd.randint(0,1)
        prob=1
        suma_social_media=local.has_fb+ local.has_ig
        if suma_social_media==2:
            prob=0.9
        elif suma_social_media==1:
            prob=0.75
        else:
            prob=0
        is_suscribed=local.is_suscribed
        sale_online= is_suscribed and rd.random()<=prob
        start_date=date.fromisoformat(range_date[0])
        end_date=date.fromisoformat(range_date[1])
        date_f=faker.date_between(start_date,end_date)
        
        list_sales.append((i,id_local,id_client,
        name_cliente,product_id,product_name,quantity_value,
        price,total_price,sale_by_publisher,sale_online,date_f,is_suscribed))

    return pd.DataFrame(list_sales,columns=label_sales)

In [28]:
def generate_services(quantity,df_locals,dogs_breed,cats_breed,range_date,services_labels,df_services):
    list_services=[]
    list_name_pets=[faker.first_name_nonbinary() for i in range (math.ceil(quantity//50))]
    list_names_clients=[faker.name() for i in range (math.ceil(quantity*0.85))]
    for i in range(quantity):
        index_local=rd.randint(0,df_locals.shape[0]-1)
        index_service=rd.randint(0,df_services.shape[0]-1)
        service=df_services.iloc[index_service]
        local=df_locals.iloc[index_local]
        suma_social_media=local.has_fb+ local.has_ig
        if suma_social_media==2:
            prob=0.9
        elif suma_social_media==1:
            prob=0.75
        else:
            prob=0
        is_suscribed=local.is_suscribed
        is_online= is_suscribed and rd.random()<=prob
        id_local_vet=local.id
        name_pet=rd.choice(list_name_pets)
        is_cat=rd.randint(0,1)
        is_dog=int(not is_cat)
        breed=''
        if is_cat:
            breed=rd.choice(cats_breed)
        else:
            breed=rd.choice(dogs_breed)
        price=service.prices
        start_date=date.fromisoformat(range_date[0])
        end_date=date.fromisoformat(range_date[1])
        date_f=faker.date_between(start_date,end_date)
        service_name=service.service_name
        sale_by_publisher=rd.randint(0,1)
        name_client=rd.choice(list_names_clients)
        
        list_services.append((i,id_local_vet,name_pet,breed,price,date_f,is_online,is_cat,is_dog,service_name,sale_by_publisher,name_client,is_suscribed))
    return pd.DataFrame(list_services,columns=services_labels)
        

In [29]:
range_price=(10,100)
range_date=('2021-05-01','2021-07-01')

In [30]:
df_product_gen=generate_products(200,pet_food,dog_breeds,cat_breeds,range_price,products_labels)
df_local_gen=generate_locals(100,locals_vets_labels)
df_sales_local_gen=generate_sales_locals(10000,df_local_gen,df_product_gen,sales_local_labels,range_date)
df_services_gen=generate_services(10000,df_local_gen[df_local_gen.is_vet==1],dog_breeds,cat_breeds,range_date,services_vet_labels,df_services)


In [31]:
df_sales_local_gen.product_name.nunique()

200

In [4]:
root_path='./data'

In [33]:
df_product_gen.to_csv(f'{root_path}/products.csv',index=False)
df_local_gen.to_csv(f'{root_path}/locals.csv',index=False)
df_sales_local_gen.to_csv(f'{root_path}/local_sales.csv',index=False)
df_services_gen.to_csv(f'{root_path}/sale_services_vet.csv',index=False)

In [7]:
df_product_gen=pd.read_csv(f'{root_path}/products_v2.csv')
df_local_gen=pd.read_csv(f'{root_path}/locals_v2.csv')
df_sales_local_gen=pd.read_csv(f'{root_path}/local_sales_v2.csv')
df_services_gen=pd.read_csv(f'{root_path}/sale_services_vet_v2.csv')

In [20]:
df_services_gen.groupby('name_client').sum().describe()

,id,id_local_vet,price,is_cat,is_dog,sale_by_publisher,is_suscribed
count,5641.000000,5641.000000,5641.000000,5641.000000,5641.000000,5641.000000,5641.000000
mean,8862.790285,96.053714,276.869349,0.895409,0.877327,0.886368,0.985109
std,6300.244528,65.691206,446.434955,0.843865,0.829115,0.828371,0.864463
min,1.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000
25%,4359.000000,48.000000,40.000000,0.000000,0.000000,0.000000,0.000000
50%,7787.000000,84.000000,70.000000,1.000000,1.000000,1.000000,1.000000
75%,11840.000000,129.000000,140.000000,1.000000,1.000000,1.000000,1.000000
max,50246.000000,614.000000,3160.000000,6.000000,6.000000,6.000000,9.000000


In [19]:
df_sales_local_gen.groupby('name_client').sum().describe()

,id,id_local,product_id,product_name,quantity,price,total_price,sale_by_publisher,is_suscribed
count,5684.000000,5684.000000,5.684000e+03,5684.000000,5684.000000,5684.000000,5684.000000,5684.000000,5684.000000
mean,8795.742435,87.296094,6.142048e+12,173.640042,9.685961,46.461843,254.998210,0.873856,0.882125
std,6271.171649,61.737974,5.896748e+12,124.403416,6.639407,39.586008,256.137264,0.821426,0.845751
min,4.000000,0.000000,1.810295e+06,0.000000,1.000000,0.051824,0.103647,0.000000,0.000000
25%,4226.250000,43.000000,8.793260e+10,84.000000,5.000000,14.777927,65.059049,0.000000,0.000000
50%,7731.500000,78.000000,5.425007e+12,152.500000,8.000000,37.208327,166.784825,1.000000,1.000000
75%,11932.750000,116.000000,8.763316e+12,231.000000,13.000000,68.629052,363.941456,1.000000,1.000000
max,51718.000000,565.000000,4.686936e+13,1004.000000,63.000000,309.154435,2108.605249,6.000000,6.000000


In [21]:
df_sales_local_gen.describe()

,id,id_local,product_id,product_name,quantity,price,total_price,sale_by_publisher,is_suscribed
count,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,49.619100,3.491140e+12,98.697000,5.505500,26.408912,144.940982,0.496700,0.501400
std,2886.89568,28.895161,3.573896e+12,57.705124,2.878263,22.723756,160.020795,0.500014,0.500023
min,0.00000,0.000000,1.810295e+06,0.000000,1.000000,0.051824,0.051824,0.000000,0.000000
25%,2499.75000,25.000000,7.365701e+10,48.000000,3.000000,8.847659,32.849228,0.000000,0.000000
50%,4999.50000,49.000000,3.182551e+12,98.000000,5.000000,18.209328,83.327588,0.000000,1.000000
75%,7499.25000,75.000000,7.613034e+12,149.000000,8.000000,38.839566,199.094646,1.000000,1.000000
max,9999.00000,99.000000,9.003579e+12,199.000000,10.000000,90.834657,908.346571,1.000000,1.000000
